# Title
[]()

In [1]:
from pandas import json_normalize  
import pandas as pd
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\custom_python")
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\src")
from silvhua import *
from datetime import datetime
# from plotly.subplots import make_subplots
# import requests
import json

In [2]:
from processing import *

In [3]:
with open("credentials.json") as f:
    credentials = json.load(f)

ig_user_id = credentials['am_ig_user_id']
access_token = credentials['am_ig_access_token']

# Load posts

In [79]:
json_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\raw'
csv_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\interim'
filename='AM_IG_posts_2022-01-01_2023-03-01.sav'
response_json_dict = loadpickle(filename, json_path)
data_processed, count_vector, vect = response_json_dict_to_df(response_json_dict)
data_processed.head()


Time completed: 2023-03-05 22:15:05.754356
Time zone: Canada/Pacific


C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\src\processing.py:37: UserWarning:

Converting to PeriodArray/Index representation will drop timezone information.

C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\src\processing.py:38: UserWarning:

Converting to PeriodArray/Index representation will drop timezone information.



Token pattern: (?u)\b\w\w+\b
Shape of vector array:  (362, 2905)

Top 10 words:
<number>    597
want        230
get         222
time        221
look        175
body        167
train       161
make        157
week        153
work        152
dtype: int64
Time processed: 2023-03-05 22:15:06.962346


,timestamp,caption,like_count,comments_count,media_type,media_product_type,media_url,permalink,id,comments.data,...,date,year,month,week_of_year,day_of_week,day_of_week_name,time,hour,year-month,year-week
0,2023-02-28 11:08:07-08:00,🎯 nutrition hierarchy importance 🎯 note st...,7,1,IMAGE,FEED,https://scontent.cdninstagram.com/v/t51.29350-...,https://www.instagram.com/p/CpN51UjB9mf/,17968352051238783,"[{'timestamp': '2023-02-28T19:35:06+0000', 'te...",...,2023-02-28,2023,2,9,1,Tuesday,11:08:07,11,2023-02-01,2023-02-27
1,2023-02-27 10:54:58-08:00,self conscious almost ruin may sound dramatic...,59,0,VIDEO,REELS,https://video.cdninstagram.com/o1/v/t16/f1/m82...,https://www.instagram.com/reel/CpLTguzhkB5/,17984097730814170,NaN,...,2023-02-27,2023,2,9,0,Monday,10:54:58,10,2023-02-01,2023-02-27
2,2023-02-26 11:21:55-08:00,devani ✅ mother ✅ radiographer originally sign...,15,1,CAROUSEL_ALBUM,FEED,https://scontent.cdninstagram.com/v/t51.29350-...,https://www.instagram.com/p/CpIx0vrBXQW/,17997080695727863,"[{'timestamp': '2023-02-26T19:44:41+0000', 'te...",...,2023-02-26,2023,2,8,6,Sunday,11:21:55,11,2023-02-01,2023-02-20
3,2023-02-23 11:00:09-08:00,‘ alone little together achieve much ’ tag pe...,19,0,IMAGE,FEED,https://scontent.cdninstagram.com/v/t51.29350-...,https://www.instagram.com/p/CpBA8mDhJ7W/,17915392925692330,NaN,...,2023-02-23,2023,2,8,3,Thursday,11:00:09,11,2023-02-01,2023-02-20
4,2023-02-22 10:54:18-08:00,deirdre hit first chin day hit two want know...,33,7,VIDEO,REELS,NaN,https://www.instagram.com/reel/Co-bX3TL7b6/,17961721733161620,"[{'timestamp': '2023-02-22T21:48:03+0000', 'te...",...,2023-02-22,2023,2,8,2,Wednesday,10:54:18,10,2023-02-01,2023-02-20


In [124]:
caption_column='caption'
data_processed.columns.str.replace(caption_column, f'post_{caption_column}')

Index(['timestamp', 'post_caption', 'like_count', 'comments_count',
       'media_type', 'media_product_type', 'media_url', 'permalink', 'id',
       'comments.data', 'thumbnail_url', 'comments.paging.cursors.after',
       'comments.paging.next', 'date', 'year', 'month', 'week_of_year',
       'day_of_week', 'day_of_week_name', 'time', 'hour', 'year-month',
       'year-week'],
      dtype='object')

In [133]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
def plot_images_tfidf(input_df, count_vector,
        n=5, top=True, max_columns=5, streamlit=False, 
        caption_column='caption', timezone='Canada/Pacific'):
    """
    Plot the images/video thumbnails of either the top or 
    worst performing instagram media (posts, reels, carousels).
    The following data is shown for each resulting media item:
        - Image/video thumbnail
        - Time stamp of the post in the provided time zone or in UTC time.
        - Number of likes and number of comments.
        - 5 words with the highest Tf-idf scores.

    Parameters:
        - df: DataFrame with the processed data.
        - count_vector: DataFrame with the count vectors.
        - n (int): Number of images to show.
        - top (bool): If True, plot images with the highest number of likes in 
            descending order. If False, plot images with the highest number of likes in
            ascending order. sort_by = ['like_count', 'comments_count', 'timestamp']
        - streamlit (bool): Whether or not the app runs on Streamlit. If False, then
            it is run on local machine.
        - timezone (str): Timezone parameter for the `.astimezone()` method.
            e.g. 'Australia/Sydney', 'Canada/Pacific'
    Returns:
        - DataFrame containing the data of the posts in the figure.
        - fig: Plotly figure object.
    """
    tfidf = tfidf_transform(count_vector)
    ncols = n if n<max_columns else max_columns
    nrows = (n + ncols - 1) // ncols
    sort_by = ['like_count', 'comments_count', 'timestamp']
    # SH 2023-03-05 21:52
    df = input_df.copy()
    df.columns = df.columns.str.replace(caption_column, f'post_{caption_column}')
    df = pd.concat([df, tfidf], axis=1)

    posts = df.sort_values(by=sort_by, ascending=False if top else True).head(n).copy()
    posts['thumbnail_url'].fillna(posts['media_url'], inplace=True)
    if timezone:
        print('Time zone:', timezone)
        converted_timestamp = [timestamp.astimezone(timezone) for timestamp in pd.to_datetime(posts['timestamp'])]
        titles = tuple(timestamp.strftime('%Y-%m-%d at %H:%M') for timestamp in converted_timestamp)
    else:
        titles = tuple(posts['timestamp'].dt.strftime('%Y-%m-%d at %H:%M').values.tolist())
    fig = make_subplots(
        rows=nrows, cols=ncols, 
        vertical_spacing=0.55/nrows
        )
    for index, (i, n_likes, n_comments, url, title) in enumerate(zip(posts.index,
            posts['like_count'], posts['comments_count'], posts['thumbnail_url'],
            titles)
        ):
        highest_tfidf = [word for word, value in tfidf.loc[i].sort_values(
            ascending=False).head(5).items() if value > 0]
        fig.add_layout_image(
            x=0, y=0,
            xanchor='center', yanchor='middle',
            sizex=1, sizey=1,
            row=index // ncols + 1,
            col=index % ncols + 1,
            xref="x",
            yref="y",
            opacity=1.0,
            source=url
        )
        annotation_y_position = -0.075
        fig.add_annotation(
            xref="x domain",
            yref="y domain",
            x=0.5,
            y=1.1,
            text=title,
            axref="x domain",
            ayref="y",
            ax=0.5,
            ay=2,
            arrowhead=2,
            row=index // ncols + 1,
            col=index % ncols + 1,
        )
        fig.add_annotation(
            xref="x domain",
            yref="y domain",
            x=0.5,
            y=annotation_y_position,
            text=f'{n_likes} likes, {n_comments} comments',
            axref="x domain",
            ayref="y",
            ax=0.5,
            ay=2,
            arrowhead=2,
            row=index // ncols + 1,
            col=index % ncols + 1,
        )
        annotation_y_position -= 0.1
        fig.add_annotation(
            xref="x domain",
            yref="y domain",
            x=0.5,
            y=annotation_y_position,
            text=f'Most unique words:',
            axref="x domain",
            ayref="y",
            ax=0.5,
            ay=2,
            arrowhead=2,
            row=index // ncols + 1,
            col=index % ncols + 1,
        )
        for word in highest_tfidf:
            annotation_y_position -= .075
            fig.add_annotation(
                xref="x domain",
                yref="y domain",
                x=0.5,
                y=annotation_y_position,
                text=word,
                axref="x domain",
                ayref="y",
                ax=0.5,
                ay=2,
                arrowhead=2,
                row=index // ncols + 1,
                col=index % ncols + 1,
                # hovertext=caption,
            )
    fig.update_xaxes(range=[-0.5,0.5], showticklabels=False)
    fig.update_yaxes(range=[-0.5,0.5], showticklabels=False)
    fig.update_layout(
        title_text='Posts with highest number of likes' if top==True else 'Posts with fewest number of likes',
        title_xanchor='center', title_x=0.5,
        plot_bgcolor="white", 
        height = 120 + nrows * 300,
        margin_b=120, margin_l=50, margin_r=50,
        )
    if streamlit:
        st.plotly_chart(fig, use_container_width=True)
    else:
        fig.show()
    return posts.reset_index(drop=True), fig


top_posts, fig = plot_images_tfidf(data_processed, count_vector)

Time zone: Canada/Pacific


In [135]:
bottom_posts, fig_bottom = plot_images_tfidf(data_processed, count_vector, top=False)

Time zone: Canada/Pacific


In [136]:
bottom_posts.loc[0, 'media_url']

'https://video.cdninstagram.com/o1/v/t16/f1/m82/43474E1AD9BC19BAEE84FA4BA76079AA_video_dashinit.mp4?efg=eyJ2ZW5jb2RlX3RhZyI6InZ0c192b2RfdXJsZ2VuLjcyMC5jbGlwcyJ9&_nc_ht=video.cdninstagram.com&_nc_cat=103&vs=1557881478059002_3784848049&_nc_vs=HBksFQIYT2lnX3hwdl9yZWVsc19wZXJtYW5lbnRfcHJvZC80MzQ3NEUxQUQ5QkMxOUJBRUU4NEZBNEJBNzYwNzlBQV92aWRlb19kYXNoaW5pdC5tcDQVAALIAQAVABgkR0Yzc3BCTk1QNGVXNGhvREFIa3hFOGZyR1d0M2JxX0VBQUFGFQICyAEAKAAYABsBiAd1c2Vfb2lsATEVAAAm2u327Kr33j8VAigCQzMsF0BWfrhR64UfGBJkYXNoX2Jhc2VsaW5lXzFfdjERAHUAAA%3D%3D&ccb=9-4&oh=00_AfCpyRXEV-WYNr1NXsRuOcHS84s80pu2bIjEshizKq69wg&oe=6401E09C&_nc_sid=ea0b6e&_nc_rid=40b9bdac3e'

In [140]:
bottom_posts['post_caption']

0    ✅ lack energy complete day day task  ✅ affect ...
1    expire tonight  invitation woman sick fad diet...
2    september strong sassy programme go better edu...
3    nicole change mindset become confident <Number...
4    expire tonight  invitation woman sick fad diet...
Name: post_caption, dtype: object

# *End of Page*